In [1]:
import os
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from tensorflow import keras
import seaborn as sns
import random
import tensorflow as tf
from sklearn.metrics import confusion_matrix

In [2]:
def find_max_wave_height(zdisp_window):
    zero_crossings = np.where(np.diff(np.sign(zdisp_window)))[0]
    zero_crossings= np.append(zero_crossings, len(zdisp_window)-1);
    zero_crossings= np.append(-1,zero_crossings)
    h_wave=np.zeros(len(zero_crossings)-1)
    t_wave=np.zeros(len(zero_crossings)-1)
            
    for iter_zero_crossing in range(len(zero_crossings)-1):
        peak_idx=np.argmax(np.abs(zdisp_window[zero_crossings[iter_zero_crossing]+1:zero_crossings[iter_zero_crossing+1]+1]))
        h_wave[iter_zero_crossing]=zdisp_window[zero_crossings[iter_zero_crossing]+1+peak_idx]
        t_wave[iter_zero_crossing]=zero_crossings[iter_zero_crossing]+1+peak_idx
                
    max_wave_height=max(np.abs(np.diff(h_wave)))
    max_index = int(t_wave[np.argmax(np.abs(np.diff(h_wave)))])
    
    return max_wave_height, max_index

In [3]:
def populate_rw_arrays(dir, array, start_idx, end_idx):
    for folder in os.listdir(dir):
        print("Processing: " + folder)
        
        start_time = time.time()
        for file in os.listdir(dir+ "/" + folder):
            if file.endswith(".npz"):
                data=np.load(dir+"/"+ folder+"/"+file)
                z_tmp=data['zdisp'][start_idx:end_idx]
                #print(z_tmp.shape)
                significant_wave_height=4*np.std(data['zdisp'])#[start_idx:-1]
                #if folder=='Buoy_067':
                #    z_disp_rw_buoy_067.append(z_tmp/significant_wave_height)
                #else:
               # z_disp_rw.append(z_tmp/significant_wave_height) 
                z_disp_rw.append(z_tmp/significant_wave_height)
                #print(z_disp_rw[0].shape)
        print("--- %s seconds ---" % (time.time() - start_time))
    return

In [4]:
def populate_norw_arrays(dir, array, end_idx):
    for folder in os.listdir(dir):
        print("Processing: " + folder)
        
        start_time = time.time()
        for file in os.listdir(dir+ "/" + folder):
            if file.endswith(".npz"):
                data=np.load(dir+"/"+ folder+"/"+file)
                z_tmp=data['zdisp_norw'][0:end_idx]
                #print(z_tmp.shape)
                significant_wave_height=4*np.std(data['zdisp_norw'])#[start_idx:-1]
                #if folder=='Buoy_067':
                #    z_disp_rw_buoy_067.append(z_tmp/significant_wave_height)
                #else:
               # z_disp_rw.append(z_tmp/significant_wave_height) 
                z_disp_norw.append(z_tmp/significant_wave_height)

        print("--- %s seconds ---" % (time.time() - start_time))
    return 

In [5]:
#num_samples_each_type = int(input("Enter the number of each type of wave sample: ")) 
#num_samples_each_type = 50000 

root_folder_rw = os.getcwd()+"/wave_height_g_2"
root_folder_norw = os.getcwd()+"/wave_height_g_2"

#root_folder="E:/"

window_start_min_rel_to_rw=-25
window_length_in_min=20

samplerate=1.28

rw_idx=round(25*60*samplerate)
start_idx=round((25+window_start_min_rel_to_rw)*60*samplerate)
end_idx=start_idx+round(window_length_in_min*60*samplerate)

rw_dir=root_folder_rw + '/rw_samples'
z_disp_rw=[] 
#z_disp_rw_buoy_067=[] 
populate_rw_arrays(rw_dir, z_disp_rw, start_idx, end_idx)

z_disp_norw=[]
#z_disp_norw_buoy_067=[]
norw_dir=root_folder_norw + '/norw_samples'
#norw_dir=root_folder_norw 
end_idx=end_idx-start_idx
populate_norw_arrays(norw_dir, z_disp_norw, end_idx)

Processing: Buoy_028
--- 3.482182264328003 seconds ---
Processing: Buoy_029
--- 6.767531871795654 seconds ---
Processing: Buoy_036
--- 6.501274108886719 seconds ---
Processing: Buoy_043
--- 2.070389986038208 seconds ---
Processing: Buoy_045
--- 3.89978289604187 seconds ---
Processing: Buoy_067
--- 4.000948190689087 seconds ---
Processing: Buoy_071
--- 4.987452507019043 seconds ---
Processing: Buoy_076
--- 5.29913592338562 seconds ---
Processing: Buoy_081
--- 0.0 seconds ---
Processing: Buoy_087
--- 0.001001119613647461 seconds ---
Processing: Buoy_088
--- 0.0 seconds ---
Processing: Buoy_089
--- 0.0 seconds ---
Processing: Buoy_090
--- 0.0009984970092773438 seconds ---
Processing: Buoy_091
--- 0.5073645114898682 seconds ---
Processing: Buoy_092
--- 4.833804368972778 seconds ---
Processing: Buoy_093
--- 1.8287203311920166 seconds ---
Processing: Buoy_094
--- 4.059160470962524 seconds ---
Processing: Buoy_095
--- 1.042621374130249 seconds ---
Processing: Buoy_096
--- 2.624063014984131 se

In [6]:
z_disp_rw=np.vstack(z_disp_rw)
#z_disp_rw_buoy_067=np.vstack(z_disp_rw_buoy_067)
print(z_disp_rw.shape)

z_disp_norw=np.vstack(z_disp_norw)
#z_disp_norw_buoy_067=np.vstack(z_disp_norw_buoy_067)

print(z_disp_norw.shape)

(169961, 1536)
(169961, 1536)


In [7]:
possible_total_waves = round(z_disp_rw.shape[0] + z_disp_norw.shape[0])
print(f"Total possible waves in our study: {possible_total_waves}")

Total possible waves in our study: 339922


In [17]:
np.random.seed(5)  
len_array_rw = len(z_disp_rw) 
len_array_norw = len(z_disp_norw)

indices_rw_train = np.random.choice(len_array_rw, round(0.8*len_array_rw), replace=False)
indices_rw_test = [ind not in indices_rw_train for ind in range(0,len_array_rw)]

indices_norw_train = np.random.choice(len_array_norw, round(0.8*len_array_norw), replace=False)
indices_norw_test = [ind not in indices_norw_train for ind in range(0,len_array_norw)]

z_disp_rw_train = z_disp_rw[indices_rw_train]
labels_rw_train = np.ones(len(z_disp_rw_train))
z_disp_rw_test = z_disp_rw[indices_rw_test]
labels_rw_test = np.ones(len(z_disp_rw_test))

z_disp_norw_train = z_disp_norw[indices_norw_train]
labels_norw_train = np.zeros(len(z_disp_norw_train))
z_disp_norw_test = z_disp_norw[indices_norw_test]
labels_norw_test = np.zeros(len(z_disp_norw_test))

In [18]:
wave_data_train = np.concatenate((z_disp_rw_train, z_disp_norw_train), axis=0)
label_train = np.concatenate((labels_rw_train, labels_norw_train), axis=0)

idx = np.random.permutation(len(wave_data_train))
wave_data_train = wave_data_train[idx]
label_train=label_train[idx]

wave_data_test = np.concatenate((z_disp_rw_test, z_disp_norw_test), axis=0)
label_test = np.concatenate((labels_rw_test, labels_norw_test), axis=0)

idx_test = np.random.permutation(len(wave_data_test))
wave_data_test = wave_data_test[idx_test]
label_test=label_test[idx_test]

wave_data_train = wave_data_train.reshape((wave_data_train.shape[0], wave_data_train.shape[1], 1))
wave_data_test = wave_data_test.reshape((wave_data_test.shape[0], wave_data_test.shape[1], 1))

num_waves_total = len(wave_data_train)+len(wave_data_test)

print(f"The total number of wave samples in the case with relative rogue wave ratio 0.5 is {num_waves_total}.", end='\n')

np.savez(os.getcwd()  +'/DataPrepared2/RWs_H_g_2_tadv_5min_rw_0.5',wave_data_train=wave_data_train, wave_data_test=wave_data_test,label_train=label_train,label_test=label_test)

The total number of wave samples in the case with relative rogue wave ratio 0.5 is 339922.


In [21]:
print(label_train.shape)
print(wave_data_train.shape)

(271938,)
(271938, 1536, 1)


In [22]:
relative_rw = [0.2, 0.3, 0.4, 0.6 ,0.7, 0.8]

for i in range(len(relative_rw)):
    ratio = relative_rw[i]
    if ratio >= 0.5:
        num_rw_train = z_disp_rw_train.shape[0]
        num_rw_test = z_disp_rw_test.shape[0]
        
        num_norw_train = round(((1-ratio)/ratio) * num_rw_train)
        num_norw_test = round(((1-ratio)/ratio) * num_rw_test)
    else:
        num_norw_train = z_disp_norw_train.shape[0]
        num_norw_test = z_disp_norw_test.shape[0]
        
        num_rw_train = round((ratio / (1-ratio))*num_norw_train)
        num_rw_test = round((ratio / (1-ratio))*num_norw_test)

    num_waves_total = num_norw_train + num_norw_test + num_rw_train + num_rw_test

    print(f"The number of training rogue wave samples in the case with relative rogue wave ratio {ratio} is {num_rw_train}.")
    print(f"The number of training non-rogue wave samples in the case with relative rogue wave ratio {ratio} is {num_norw_train}.")
    print(f"The number of testing rogue wave samples in the case with relative rogue wave ratio {ratio} is {num_rw_test}.")
    print(f"The number of testing non-rogue wave samples in the case with relative rogue wave ratio {ratio} is {num_norw_test}.")
    print(f"The total number of wave samples in the case with relative rogue wave ratio {ratio} is {num_waves_total}.", end='\n\n\n')

    np.random.seed(5)  
    len_array_rw_train = len(z_disp_rw_train) 
    len_array_norw_train = len(z_disp_norw_train)

    len_array_rw_test = len(z_disp_rw_test) 
    len_array_norw_test = len(z_disp_norw_test)

    indices_rw_train = np.random.choice(len_array_rw_train, num_rw_train, replace=False)
    indices_rw_test = np.random.choice(len_array_rw_test, num_rw_test, replace=False)

    indices_norw_train = np.random.choice(len_array_norw_train, num_norw_train, replace=False)
    indices_norw_test = np.random.choice(len_array_norw_test, num_norw_test, replace=False)

    z_disp_rw_train_altered = z_disp_rw_train[indices_rw_train]
    labels_rw_train_altered = np.ones(len(z_disp_rw_train_altered))
    z_disp_rw_test_altered = z_disp_rw_test[indices_rw_test]
    labels_rw_test_altered = np.ones(len(z_disp_rw_test_altered))

    z_disp_norw_train_altered = z_disp_norw_train[indices_norw_train]
    labels_norw_train_altered = np.zeros(len(z_disp_norw_train_altered))
    z_disp_norw_test_altered = z_disp_norw_test[indices_norw_test]
    labels_norw_test_altered = np.zeros(len(z_disp_norw_test_altered))

    # np.random.seed(5)  
    # len_array_rw = len(z_disp_rw) 
    # len_array_norw = len(z_disp_norw)

    # indices_rw = np.random.choice(len_array_rw, num_rw, replace=False) 
    # indices_norw = np.random.choice(len_array_norw, num_norw, replace=False)

    # z_disp_rw_altered = z_disp_rw[indices_rw] 
    # z_disp_norw_altered = z_disp_norw[indices_norw]

    wave_data_train = np.concatenate((z_disp_rw_train_altered, z_disp_norw_train_altered), axis=0)
    label_train = np.concatenate((labels_rw_train_altered, labels_norw_train_altered), axis=0)

    idx = np.random.permutation(len(wave_data_train))
    wave_data_train = wave_data_train[idx]
    label_train=label_train[idx]

    wave_data_test = np.concatenate((z_disp_rw_test_altered, z_disp_norw_test_altered), axis=0)
    label_test = np.concatenate((labels_rw_test_altered, labels_norw_test_altered), axis=0)

    idx_test = np.random.permutation(len(wave_data_test))
    wave_data_test = wave_data_test[idx_test]
    label_test=label_test[idx_test]

    wave_data_train = wave_data_train.reshape(wave_data_train.shape[0], wave_data_train.shape[1], 1)
    wave_data_test = wave_data_test.reshape(wave_data_test.shape[0], wave_data_test.shape[1], 1)

    print(label_train.shape)
    print(wave_data_train.shape)
    print(label_test.shape)
    print(wave_data_test.shape)

    np.savez(os.getcwd()  +f'/DataPrepared2/RWs_H_g_2_tadv_5min_rw_{relative_rw[i]}',wave_data_train=wave_data_train, wave_data_test=wave_data_test,label_train=label_train,label_test=label_test)

The number of training rogue wave samples in the case with relative rogue wave ratio 0.2 is 33992.
The number of training non-rogue wave samples in the case with relative rogue wave ratio 0.2 is 135969.
The number of testing rogue wave samples in the case with relative rogue wave ratio 0.2 is 8498.
The number of testing non-rogue wave samples in the case with relative rogue wave ratio 0.2 is 33992.
The total number of wave samples in the case with relative rogue wave ratio 0.2 is 212451.


(169961,)
(169961, 1536, 1)
(42490,)
(42490, 1536, 1)
The number of training rogue wave samples in the case with relative rogue wave ratio 0.3 is 58272.
The number of training non-rogue wave samples in the case with relative rogue wave ratio 0.3 is 135969.
The number of testing rogue wave samples in the case with relative rogue wave ratio 0.3 is 14568.
The number of testing non-rogue wave samples in the case with relative rogue wave ratio 0.3 is 33992.
The total number of wave samples in the case wit